In [1]:
from ctapipe.io.hessio import hessio_event_source
from ctapipe.io import event_source

from ctapipe.calib.camera import calibrator
import sys
import os
from os.path import expandvars

import numpy as np
import h5py
import multiprocessing
from datetime import datetime
from ctapipe.image import NullWaveformCleaner, CHECMWaveformCleanerAverage, CHECMWaveformCleanerLocal
from ctapipe.image import FullIntegrator, SimpleIntegrator, GlobalPeakIntegrator, LocalPeakIntegrator, NeighbourPeakIntegrator, AverageWfPeakIntegrator
from ctapipe.image import tailcuts_clean, hillas_parameters
from ctapipe.reco import HillasReconstructor
from ctapipe.utils import datasets, linalg
from astropy import units as u
#from tqdm import tqdm

from datetime import datetime as time

In [2]:
from konsta_cta.readdata import FileReader

In [3]:
protons = FileReader("proton", directory="~/CTAPIPE/")
electron = FileReader("electron", directory="~/CTAPIPE/")
gamma_cone = FileReader("gamma_cone", directory="~/CTAPIPE/")
gamma_onSource = FileReader("gamma_onSource", directory="~/CTAPIPE/")

In [4]:
protons.read_files()

In [5]:
for event in protons.sources[0]:
    break
event

ctapipe.io.containers.DataContainer:
                          r0.*: Raw Data
                          r1.*: R1 Calibrated Data
                         dl0.*: DL0 Data Volume Reduced Data
                         dl1.*: DL1 Calibrated image
                         dl2.*: Reconstructed Shower Information
                          mc.*: Monte-Carlo data
                    mcheader.*: Monte-Carlo run header data
                        trig.*: central trigger information
                         count: number of events processed
                        inst.*: instrumental information (deprecated
                   pointing[*]: Telescope pointing positions

In [6]:
class event_information():
    def __init__(self):
        pass
    
    def collect_information(self):
        pass
    